In [1]:
import pandas as pd
import numpy as np
from itertools import product
import h5py

In [2]:
transactions = pd.read_csv('sales_train.csv')

In [3]:
index_cols=['shop_id','item_id','date_block_num']
all_shops=transactions.shop_id.drop_duplicates()
all_items=transactions.item_id.drop_duplicates()
all_dates=transactions.date_block_num.drop_duplicates().to_list()
all_dates.append(34)
grid=np.array(list(product(*[all_shops,all_items,all_dates])),dtype=np.int32)

In [4]:
prev_sales=pd.DataFrame(grid,columns=index_cols,dtype=np.int32)

In [5]:
# prev_sales.to_pickle('prev_sales.pkl')

In [6]:
# prev_sales=pd.read_pickle('prev_sales.pkl')

In [7]:
prev_sales.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,22154,1
2,59,22154,2
3,59,22154,3
4,59,22154,4


In [8]:
new_data=pd.read_csv('new_data.csv')
new_data.head()

,shop_id,item_id,date_block_num,median_price,item_cnt_month,City,Type,item_category_id,main_cat,month,lag_median_price,lag_item_cnt_month,lag_shop_sales,lag_item_sales
0,0,12,1,0.0,0.0,0,0,55,2,2,0.0,0.0,5578.0,0.0
1,0,19,0,0.0,0.0,0,0,40,0,1,0.0,0.0,0.0,0.0
2,0,27,0,0.0,0.0,0,0,19,1,1,0.0,0.0,0.0,0.0
3,0,27,1,0.0,0.0,0,0,19,1,2,0.0,0.0,5578.0,7.0
4,0,28,0,0.0,0.0,0,0,30,1,1,0.0,0.0,0.0,0.0


In [9]:
index_cols=['shop_id','item_id','date_block_num']
prev_sales=prev_sales.merge(new_data[['shop_id','item_id','date_block_num','median_price','item_cnt_month']],on=index_cols,how='left')
prev_sales

,shop_id,item_id,date_block_num,median_price,item_cnt_month
0,59,22154,0,999.0,1.0
1,59,22154,1,0.0,0.0
2,59,22154,2,0.0,0.0
3,59,22154,3,0.0,0.0
4,59,22154,4,NaN,NaN
...,...,...,...,...,...
45794695,36,7640,30,NaN,NaN
45794696,36,7640,31,NaN,NaN
45794697,36,7640,32,NaN,NaN
45794698,36,7640,33,0.0,0.0


In [10]:
prev_sales=prev_sales.fillna(0)
prev_sales['revenue']=prev_sales.median_price*prev_sales.item_cnt_month
prev_sales

,shop_id,item_id,date_block_num,median_price,item_cnt_month,revenue
0,59,22154,0,999.0,1.0,999.0
1,59,22154,1,0.0,0.0,0.0
2,59,22154,2,0.0,0.0,0.0
3,59,22154,3,0.0,0.0,0.0
4,59,22154,4,0.0,0.0,0.0
...,...,...,...,...,...,...
45794695,36,7640,30,0.0,0.0,0.0
45794696,36,7640,31,0.0,0.0,0.0
45794697,36,7640,32,0.0,0.0,0.0
45794698,36,7640,33,0.0,0.0,0.0


In [11]:
prev_sales['revenue_lag_1']=prev_sales.groupby(['shop_id','item_id'])['revenue'].shift(1)

In [12]:
prev_sales['revenue_lag_2']=prev_sales.groupby(['shop_id','item_id'])['revenue'].shift(2)

In [13]:
prev_sales['revenue_lag_3']=prev_sales.groupby(['shop_id','item_id'])['revenue'].shift(3)

In [14]:
prev_sales['revenue_lag_6']=prev_sales.groupby(['shop_id','item_id'])['revenue'].shift(6)

In [15]:
prev_sales['revenue_lag_9']=prev_sales.groupby(['shop_id','item_id'])['revenue'].shift(9)

In [16]:
prev_sales['revenue_lag_12']=prev_sales.groupby(['shop_id','item_id'])['revenue'].shift(12)

In [17]:
prev_sales

,shop_id,item_id,date_block_num,median_price,item_cnt_month,revenue,revenue_lag_1,revenue_lag_2,revenue_lag_3,revenue_lag_6,revenue_lag_9,revenue_lag_12
0,59,22154,0,999.0,1.0,999.0,NaN,NaN,NaN,NaN,NaN,NaN
1,59,22154,1,0.0,0.0,0.0,999.0,NaN,NaN,NaN,NaN,NaN
2,59,22154,2,0.0,0.0,0.0,0.0,999.0,NaN,NaN,NaN,NaN
3,59,22154,3,0.0,0.0,0.0,0.0,0.0,999.0,NaN,NaN,NaN
4,59,22154,4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
45794695,36,7640,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45794696,36,7640,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45794697,36,7640,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45794698,36,7640,33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
prev_sales.drop('item_cnt_month',axis=1,inplace=True)

In [27]:
prev_sales.to_pickle('prev_revenue.csv')

MemoryError: 